# Wikipedia Music Genres NLP Project

## Overview

This notebook performs an NLP on music articles collected from [Wikipedia](https://en.wikipedia.org/).

## Imports

In [ ]:
import pandas as pd

## Data Collection

In [ ]:
# Wiki API and Web Scraping

## Data Preprocessing

### Tokenize

### Remove stopwords

### Lemmatize or apply stemming

## Feature Extraction

In [ ]:
# TF-IDF, BERT embeddings
# Word embeddings like Word2Vec, Glove

In [ ]:
# Or use HF feature extraction model that does all those steps

## Unsupervised Learning

### K-Means

### DBSCAN

### Latent Dirichlet Allocation (LDA)?

## Evaluation and Interpretation

### Visualize Clusters

In [ ]:
#  t-SNE, UMAP, PCA

### Analyse Clusters

## End